In [1]:
from braincoder.hrf import SPMHRFModel
from braincoder.models import GaussianPRF2DWithHRF, DifferenceOfGaussiansPRF2DWithHRF, DivisiveNormalizationGaussianPRF2DWithHRF

from braincoder.utils.data import load_szinte2024
from braincoder.optimize import ParameterFitter

import numpy as np

data = load_szinte2024()
paradigm = data['stimulus']
grid_coordinates = data['grid_coordinates']
train_data = data['v1_timeseries'].iloc[:, :200]
train_data.index.name = 'frame'
TR = data['tr']

/Users/gdehol/mambaforge/envs/braincoder/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hrf_model = SPMHRFModel(tr=TR)
positive_amplitude = True

debug = False

if debug:
    min_n_iterations = 10
    max_n_iterations = 100
    train_data = train_data.iloc[:, :5]
else:
    min_n_iterations = 100
    max_n_iterations = 1000


grid_parameters = {'x':np.linspace(-5, 5, 10),
                   'y':np.linspace(-5, 5, 10),
                   'size':np.linspace(1, 3, 10)}

model_gauss = GaussianPRF2DWithHRF(data=train_data,
                                    paradigm=paradigm,
                                    hrf_model=hrf_model,
                                    grid_coordinates=grid_coordinates)
par_fitter = ParameterFitter(model=model_gauss, data=train_data, paradigm=paradigm)
baseline = [0.0]
amplitude = [1.0]
pars_gauss_grid = par_fitter.fit_grid(grid_parameters['x'],
                                        grid_parameters['y'],
                                        grid_parameters['size'],
                                        baseline,
                                        amplitude,
                                        correlation_cost=True)
pars_gauss_ols = par_fitter.refine_baseline_and_amplitude(pars_gauss_grid,
                                                            positive_amplitude=positive_amplitude)
pars_gauss_gd = par_fitter.fit(init_pars=pars_gauss_ols,
                                min_n_iterations=min_n_iterations,
                                max_n_iterations=max_n_iterations)

model_hrf = GaussianPRF2DWithHRF(data=train_data,
                                    paradigm=paradigm,
                                    hrf_model=hrf_model,
                                    grid_coordinates=grid_coordinates,
                                    flexible_hrf_parameters=True)

par_fitter_hrf = ParameterFitter(model=model_hrf, data=train_data, paradigm=paradigm)

# We set hrf_delay and hrf_dispersion to standard values
pars_gauss_gd['hrf_delay'] = 6
pars_gauss_gd['hrf_dispersion'] = 1

pars_gauss_hrf = par_fitter_hrf.fit(init_pars=pars_gauss_gd,
                                    min_n_iterations=min_n_iterations,
                                    max_n_iterations=max_n_iterations)
pars_gauss_hrf['r2'] = par_fitter_hrf.get_rsq(pars_gauss_hrf)

model_dog = DifferenceOfGaussiansPRF2DWithHRF(data=train_data,
                                                paradigm=paradigm,
                                                hrf_model=hrf_model,
                                                grid_coordinates=grid_coordinates,
                                                flexible_hrf_parameters=True)

pars_dog_init = pars_gauss_hrf.copy()
pars_dog_init['amplitude'] = np.clip(pars_dog_init['amplitude'], 1e-4, None)
# This is the relative amplitude of the inhibitory receptive field
# compared to the excitatory one.
pars_dog_init['srf_amplitude'] = 0.0001

# This is the relative size of the inhibitory receptive field
# compared to the excitatory one.
pars_dog_init['srf_size'] = 4.
pars_dog_init = pars_dog_init.drop(columns=['r2'])

# Let's set up a new parameterfitter
par_fitter_dog = ParameterFitter(model=model_dog, data=train_data, paradigm=paradigm)

# Note how, for now, we are not optimizing the HRF parameters.
pars_dog_hrf = par_fitter_dog.fit(init_pars=pars_dog_init,
                                min_n_iterations=min_n_iterations,
                                max_n_iterations=max_n_iterations)

Working with chunk size of 22222


100%|██████████| 1/1 [00:00<00:00,  1.12it/s]


*** Fitting: ***
 * x
 * y
 * sd
 * baseline
 * amplitude
Number of problematic voxels (mask): 3
Number of voxels remaining (mask): 197


Current R2: 0.13165/Best R2: 0.13166: 100%|██████████| 1000/1000 [00:23<00:00, 42.89it/s]


*** Fitting: ***
 * x
 * y
 * sd
 * baseline
 * amplitude
 * hrf_delay
 * hrf_dispersion
Number of problematic voxels (mask): 3
Number of voxels remaining (mask): 197


  0%|          | 0/1000 [00:00<?, ?it/s]

(1, 150, 200) (24, 200)


Current R2: 0.24981/Best R2: 0.24981: 100%|██████████| 1000/1000 [00:23<00:00, 41.74it/s]


*** Fitting: ***
 * x
 * y
 * sd
 * baseline
 * amplitude
 * srf_amplitude
 * srf_size
 * hrf_delay
 * hrf_dispersion
Number of problematic voxels (mask): 3
Number of voxels remaining (mask): 197


Current R2: 0.27858/Best R2: 0.27875: 100%|██████████| 1000/1000 [00:33<00:00, 29.42it/s]


In [3]:
dn_model = DivisiveNormalizationGaussianPRF2DWithHRF(data=train_data,
                                                     paradigm=paradigm,
                                                     hrf_model=hrf_model,
                                                     grid_coordinates=grid_coordinates,
                                                     flexible_hrf_parameters=True)

# dn_optimizer = ParameterFitter(model=dn_model, data=train_data, paradigm=paradigm)

dn_model.parameter_labels

dn_init_pars = pars_dog_hrf.copy().rename(columns={'amplitude':'rf_amplitude'})

dn_init_pars['neural_baseline'] = 0.1
dn_init_pars['surround_baseline'] = 0.1
dn_init_pars['bold_baseline'] = 0.0

dn_init_pars = dn_init_pars.astype(np.float32)[dn_model.parameter_labels]


dn_fitter = ParameterFitter(model=dn_model, data=train_data, paradigm=paradigm)

dn_pars = dn_fitter.fit(init_pars=dn_init_pars, min_n_iterations=min_n_iterations, max_n_iterations=max_n_iterations)

*** Fitting: ***
 * x
 * y
 * sd
 * rf_amplitude
 * srf_amplitude
 * srf_size
 * neural_baseline
 * surround_baseline
 * bold_baseline
 * hrf_delay
 * hrf_dispersion
Number of problematic voxels (mask): 3
Number of voxels remaining (mask): 197


Current R2: 0.21789/Best R2: 0.27152: 100%|██████████| 1000/1000 [00:38<00:00, 26.05it/s] 
/Users/gdehol/git/braincoder/braincoder/models.py:1724: RuntimeWarning: invalid value encountered in divide
  pre_convolve = pre_convolve - (neural_baseline / surround_baseline)


In [4]:
dn_pars

parameter,x,y,sd,rf_amplitude,srf_amplitude,srf_size,neural_baseline,surround_baseline,bold_baseline,hrf_delay,hrf_dispersion
source,,,,,,,,,,,
0,1.920845,-4.245225,0.053340,12.300174,0.021438,7.523510,0.274896,0.029557,0.026175,7.317585,2.954388
1,4.064047,-1.936402,2.522991,0.006049,0.000020,1.620529,0.113666,0.115278,-0.163056,3.500023,0.500016
2,5.652807,-2.837129,2.834955,0.012332,0.000019,1.675583,0.115178,0.116250,-0.287560,3.500017,0.500036
3,6.090886,-2.341196,2.170830,0.008086,0.000023,1.833226,0.114170,0.115373,-0.170178,3.500017,0.500018
4,14.822905,-9.286911,5.711717,0.183721,0.062646,2.279780,0.142791,0.152810,-0.195409,3.500036,0.500035
...,...,...,...,...,...,...,...,...,...,...,...
195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
196,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
197,3.267166,-0.264973,3.459649,0.002314,0.449181,10.536447,0.136259,0.115644,-0.069686,3.501955,0.500946
